In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [24]:
# Dataset preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10
dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split train set into train and validation
train_idx, val_idx = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
# CNN Model
def create_cnn_model(kernel_size, pooling_type):
    class CNN(nn.Module):
        def __init__(self):
            super(CNN, self).__init__()
            self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, stride=1, padding=kernel_size//2)
            self.pool = pooling_type(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, stride=1, padding=kernel_size//2)
            self.fc1 = nn.Linear(64 * 8 * 8, 512)
            self.fc2 = nn.Linear(512, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 64 * 8 * 8)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    return CNN()

In [26]:
# Training and evaluation
def train_and_evaluate(model, optimizer_name, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_name == "RMSProp":
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    early_stop_counter = 0
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        accuracy = 100 * correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= 10:
            print("Early stopping triggered.")
            break

    return model


In [27]:
# Configurations
kernel_sizes = [3, 5, 7]
pooling_types = [nn.MaxPool2d, nn.AvgPool2d]
num_epochs_list = [5, 50, 100, 250, 350]
optimizers = ["SGD", "RMSProp", "Adam"]

# Running experiments
for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer_name in optimizers:
            for num_epochs in num_epochs_list:
                print(f"\n--- Kernel Size: {kernel_size}, Pooling: {pooling_type.__name__}, Optimizer: {optimizer_name}, Epochs: {num_epochs} ---")
                model = create_cnn_model(kernel_size, pooling_type)
                trained_model = train_and_evaluate(model, optimizer_name, num_epochs)


--- Kernel Size: 3, Pooling: MaxPool2d, Optimizer: SGD, Epochs: 5 ---
Epoch 1/5, Loss: 987.3135, Val Loss: 1.2325, Accuracy: 55.41%
Epoch 2/5, Loss: 713.8624, Val Loss: 1.0298, Accuracy: 63.89%
Epoch 3/5, Loss: 583.3067, Val Loss: 0.9352, Accuracy: 66.74%
Epoch 4/5, Loss: 486.3772, Val Loss: 0.9350, Accuracy: 67.62%
Epoch 5/5, Loss: 391.9651, Val Loss: 0.8907, Accuracy: 69.50%

--- Kernel Size: 3, Pooling: MaxPool2d, Optimizer: SGD, Epochs: 50 ---
Epoch 1/50, Loss: 987.5298, Val Loss: 1.2472, Accuracy: 56.02%
Epoch 2/50, Loss: 713.0110, Val Loss: 1.0285, Accuracy: 64.50%
Epoch 3/50, Loss: 577.7405, Val Loss: 0.9268, Accuracy: 67.58%
Epoch 4/50, Loss: 477.8185, Val Loss: 0.8429, Accuracy: 70.55%
Epoch 5/50, Loss: 383.1876, Val Loss: 0.8533, Accuracy: 70.22%
Epoch 6/50, Loss: 287.5130, Val Loss: 0.8882, Accuracy: 71.55%
Epoch 7/50, Loss: 202.8574, Val Loss: 0.9628, Accuracy: 72.24%
Epoch 8/50, Loss: 126.9291, Val Loss: 1.1906, Accuracy: 69.63%
Epoch 9/50, Loss: 77.0346, Val Loss: 1.1836